In [1]:
from tfx import v1 as tfx
import os
from absl import logging
import tensorflow as tf

In [10]:
 #Setting up some filepath constants
 PIPELINE_NAME = "sms-spam-pipeline"
 PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
 METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
 SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)
 module_file = 'trainer.py'

In [4]:
def make_dir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

In [5]:
make_dir(PIPELINE_ROOT)
make_dir(SERVING_MODEL_DIR)
make_dir(os.path.join('metadata', PIPELINE_NAME))

In [6]:
logging.set_verbosity(logging.INFO)

In [7]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext()

In [60]:
example_gen = tfx.components.CsvExampleGen(input_base='/home/gerardo/Desktop/Projects/Kubeflow-DigitalOcean/pipeline/experiments/data/clean')
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 20
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 53
        type_id: 14
        uri: "/tmp/tfx-interactive-2021-12-13T20_47_34.492322-0l515zi9/CsvExampleGen/examples/12"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:467368,xor_checksum:1638689375,sum_checksum:1638689375"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1639535210961
        last_update_time_since_epoch: 1639535212416
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [69]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  print(example)

features {
  feature {
    key: "label"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "text"
    value {
      bytes_list {
        value: "Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
      }
    }
  }
}



In [61]:
statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 21
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 54
        type_id: 16
        uri: "/tmp/tfx-interactive-2021-12-13T20_47_34.492322-0l515zi9/StatisticsGen/statistics/13"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1639535217830
        last_update_time_since_epoch: 1639535220261
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [62]:
schema_gen = tfx.components.SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 22
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 55
        type_id: 18
        uri: "/tmp/tfx-interactive-2021-12-13T20_47_34.492322-0l515zi9/SchemaGen/schema/14"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1639535221733
        last_update_time_since_epoch: 1639535221764
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [92]:
transform = tfx.components.Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_gen.outputs['schema'],
      materialize=False,
      module_file=module_file)
context.run(transform)

INFO:absl:Generating ephemeral wheel package for '/home/gerardo/Desktop/Projects/Kubeflow-DigitalOcean/pipeline/experiments/trainer.py' (including modules: ['trainer']).
INFO:absl:User module package has hash fingerprint version 879b2e46df6c14c0a09aac76d7b352410685c63ac86d7fdcb1bf5ea0ac1cae3b.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmpsem1qys2/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp8j1lv_f8', '--dist-dir', '/tmp/tmplz0xgha_']
INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2021-12-13T20_47_34.492322-0l515zi9/_wheels/tfx_user_code_Transform-0.0+879b2e46df6c14c0a09aac76d7b352410685c63ac86d7fdcb1bf5ea0ac1cae3b-py3-none-any.whl'; target user module is 'trainer'.
INFO:absl:Full user module path is 'trainer@/tmp/tfx-interactive-2021-12-13T20_47_34.492322-0l515zi9/_wheels/tfx_user_code_Transform-0.0+879b2e46df6c14c0a09aac76d7b352410685c63ac86d7fdcb1bf5ea0ac1cae3b-py3-none-any.whl'
INFO:absl:Running driver for Transform
INFO

TypeError: Tensors in list passed to 'values' of 'ConcatV2' Op have types [int64, string] that don't all match.

In [72]:
module_file = 'trainer.py'
trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      transform_graph=transform.outputs['transform_graph'],
      train_args=tfx.proto.TrainArgs(num_steps=10),
      eval_args=tfx.proto.EvalArgs(num_steps=3))
context.run(trainer)

INFO:absl:Generating ephemeral wheel package for '/home/gerardo/Desktop/Projects/Kubeflow-DigitalOcean/pipeline/experiments/trainer.py' (including modules: ['trainer']).
INFO:absl:User module package has hash fingerprint version a0a13977e0fafd183a41c93128534df602b555c593dbf70386ed5cf0c0aba81f.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmpfkax6b46/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmponxbjnad', '--dist-dir', '/tmp/tmpfmb0i377']
INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2021-12-13T20_47_34.492322-0l515zi9/_wheels/tfx_user_code_Trainer-0.0+a0a13977e0fafd183a41c93128534df602b555c593dbf70386ed5cf0c0aba81f-py3-none-any.whl'; target user module is 'trainer'.
INFO:absl:Full user module path is 'trainer@/tmp/tfx-interactive-2021-12-13T20_47_34.492322-0l515zi9/_wheels/tfx_user_code_Trainer-0.0+a0a13977e0fafd183a41c93128534df602b555c593dbf70386ed5cf0c0aba81f-py3-none-any.whl'
INFO:absl:Running driver for Trainer
INFO:absl:

Epoch 1/100


UnimplementedError:  Cast string to float is not supported
	 [[node sequential/Cast (defined at /home/gerardo/Desktop/Projects/Kubeflow-DigitalOcean/pipeline/experiments/trainer.py:99) ]] [Op:__inference_train_function_3842]

Function call stack:
train_function


In [ ]:
pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=SERVING_MODEL_DIR)))
context.run(pusher)